# Using Ray to Execute Parellel Code

For more details about Ray refer to <https://docs.ray.io/en/latest/ray-core/walkthrough.html>.

In [1]:
import ray
import time

Initialize a local Ray cluster.

In [2]:
ray.init()

2023-01-12 18:03:09,349	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=0.49gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-01-12 18:03:09,552	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.4
Ray version:,2.2.0


Define two functions that do the same thing. The first function does not use Ray, each iteration will run one after the other. The second function uses Ray. The iterations can run in parellel.

In [3]:
def normal_function(x):
    print(f"Iteration {x}")
    time.sleep(1)
    return x

@ray.remote
def ray_function(x):
    print(f"Iteration {x}")
    time.sleep(1)
    return x

number_of_iterations = 5

Test how long it takes to run the function in a **single thread/process**.

In [4]:
%%time
values = [normal_function(i) for i in range(number_of_iterations)]
print(values)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
[0, 1, 2, 3, 4]
CPU times: user 38.2 ms, sys: 1.88 ms, total: 40.1 ms
Wall time: 5.01 s


Test how long it takes to run the function using **Ray**.

In [5]:
%%time
futures = [ray_function.remote(i) for i in range(number_of_iterations)]
values = ray.get(futures)
print(values)

(ray_function pid=619) Iteration 2
(ray_function pid=618) Iteration 1
(ray_function pid=617) Iteration 0
(ray_function pid=619) Iteration 4
(ray_function pid=618) Iteration 3
[0, 1, 2, 3, 4]
CPU times: user 24.5 ms, sys: 14.1 ms, total: 38.6 ms
Wall time: 2.05 s
